# Universal Bank -WE04 

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# set random seed to ensure that results are repeatable
np.random.seed(1)


# 2.0 Loading data

In [5]:
ubank=pd.read_csv("UniversalBank.csv")

# Check the missing values by summing the total na's for each variable

In [6]:
ubank.isna().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

# Dropping least important columns

In [7]:
ubank.drop(ubank.columns[[0,4]], axis=1, inplace=True)

# Data split -train and test

In [8]:
# split the data into validation and training set
train_df, test_df = train_test_split(ubank, test_size=0.3,random_state=1)

# to reduce repetition in later code, create variables to represent the columns
# that are our predictors and target
target = 'CD Account'
predictors = list(ubank.columns)
predictors.remove(target)


In [9]:
# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
scaler.fit(train_df[predictors])

# Transform the predictors of training and test sets
train_X = scaler.transform(train_df[predictors]) 
train_y = train_df[target] 

test_X = scaler.transform(test_df[predictors])
test_y = test_df[target] 

# Logistic Regression using RandomSearch and Grid Search

In [13]:
score_measure = "recall"
kfolds = 5

param_grid = {'C':[0.001,0.01,0.1,1,10], # C is the regulization strength
               'penalty':['l1', 'l2','elasticnet','none'],
              'solver':['saga','liblinear'],
              'max_iter': np.arange(500,1000)
    
    
}

lr = LogisticRegression()
rand_search = RandomizedSearchCV(estimator = lr, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestlr = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.6708245243128964
... with parameters: {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 851, 'C': 0.1}


C:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
910 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
280 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 64, in _check_solver
    raise ValueError(
ValueError: Only 'saga' solver supports elasticnet pe

In [14]:
score_measure = "recall"
kfolds = 5
best_penality = rand_search.best_params_['penalty']
best_solver = rand_search.best_params_['solver']
min_regulization_strength=rand_search.best_params_['C']
min_iter = rand_search.best_params_['max_iter']

#Using the best parameters from the Random Search to use as range for the parameters to do the grid search
param_grid = {
    
    'C':np.arange(min_regulization_strength-1,min_regulization_strength+1), 
               'penalty':[best_penality],
              'solver':[best_solver],
              'max_iter': np.arange(min_iter-400,min_iter+400)
}

logreg =  LogisticRegression()
grid_search = GridSearchCV(estimator = logreg, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1, # n_jobs=-1 will utilize all available CPUs 
                return_train_score=True)

_ = grid_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestlogreg = grid_search.best_estimator_

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits
The best recall score is 0.6708245243128964
... with parameters: {'C': 0.09999999999999998, 'max_iter': 451, 'penalty': 'l2', 'solver': 'liblinear'}


C:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
4000 fits failed out of a total of 8000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4000 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 1160, in fit
    self._validate_params()
  File "C:\Anaconda\lib\site-packages\sklearn\base.py", line 581, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda\lib\site-packages\sklearn\utils\_param_validation.py", line 97, in v

In [15]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
Recall_lr={TP/(TP+FN)}

Accuracy=0.9780000 Precision=1.0000000 Recall=0.6024096 F1=0.7518797


In [26]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic using random & grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic using random & grid search,0.974,0.866667,0.626506,0.727273


# SVM

In [17]:
#Random search
score_measure = "recall"
kfolds = 3

param_grid = {'C':np.arange(0.1,50,10),  #  regularization parameter.
               'kernel':['linear', 'rbf','poly'],
              'gamma':['scale','auto'],
              'degree':np.arange(1,10), #degree is for the polynomial kernal
              'coef0':np.arange(1,10) #coef0 is for the polynomial kernal
                  
}

    

SVM_R_out = SVC()
rand_search = RandomizedSearchCV(estimator = SVM_R_out, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestSVMrandom = rand_search.best_estimator_

Fitting 3 folds for each of 500 candidates, totalling 1500 fits
The best recall score is 0.7351598173515982
... with parameters: {'kernel': 'poly', 'gamma': 'scale', 'degree': 4, 'coef0': 9, 'C': 20.1}


In [18]:
#grid search 
score_measure = "recall"
kfolds = 3
best_kernel = rand_search.best_params_['kernel']
best_gamma = rand_search.best_params_['gamma']
min_regulization=rand_search.best_params_['C']
best_degree = rand_search.best_params_['degree']
best_coef0=rand_search.best_params_['coef0']

#Using the best parameters from the Random Search to use as range for the parameters to do the grid search
param_grid = {
    
    'C':np.arange(min_regulization-3,min_regulization+3), 
               'kernel':[best_kernel],
              'gamma':[best_gamma],
              'degree': np.arange(best_degree-1,best_degree+1),
            'coef0': np.arange(best_coef0-3,best_coef0+3)
}

SVM_G_out = SVC()
grid_search = GridSearchCV(estimator = SVM_G_out, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestSVMgrid = grid_search.best_estimator_

Fitting 3 folds for each of 72 candidates, totalling 216 fits
The best recall score is 0.7397260273972602
... with parameters: {'C': 17.1, 'coef0': 11, 'degree': 4, 'gamma': 'scale', 'kernel': 'poly'}


In [19]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
Recall_svm={TP/(TP+FN)}

Accuracy=0.9560000 Precision=0.5934066 Recall=0.6506024 F1=0.6206897


In [20]:
performance= pd.concat([performance, pd.DataFrame({'model':"svm using Random & Grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,logistic using random & grid search,0.978,1.000000,0.602410,0.75188
0,svm using Random & Grid search,0.956,0.593407,0.650602,0.62069


#  Decision Tree classifier using RandomSearch and GridSearch

In [21]:
#decision tree classifier
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,42),  
    'min_samples_leaf': np.arange(1,42),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 42), 
    'max_depth': np.arange(1,42), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.6938689217758987
... with parameters: {'min_samples_split': 1, 'min_samples_leaf': 3, 'min_impurity_decrease': 0.0001, 'max_leaf_nodes': 38, 'max_depth': 18, 'criterion': 'gini'}


In [22]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(3,6),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0009, 0.0012,0.0001),
    'max_leaf_nodes': np.arange(36,40), 
    'max_depth': np.arange(39,42), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
The best recall score is 0.6893234672304439
... with parameters: {'criterion': 'entropy', 'max_depth': 39, 'max_leaf_nodes': 39, 'min_impurity_decrease': 0.001, 'min_samples_leaf': 3, 'min_samples_split': 5}


In [25]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
Recall_lr={TP/(TP+FN)}

performance= pd.concat([performance, pd.DataFrame({'model':"Decision tree using Random & Grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

Accuracy=0.9740000 Precision=0.8666667 Recall=0.6265060 F1=0.7272727


,model,Accuracy,Precision,Recall,F1
0,SSVm using random & grid search,0.956,0.593407,0.650602,0.620690
0,Decision tree using Random & Grid search,0.974,0.866667,0.626506,0.727273
0,Decision tree using Random & Grid search,0.974,0.866667,0.626506,0.727273


Initially, as a part of data cleansing process, I checked the missing values and found that there are no missing values and since there are no categorical variables, I havent done any encoding. Atlast removed two columns named ID and ZIP code as they are least important and then split the data into train and test.

By applying 'recall' as the score measure and using Random & grid search method to find the best fit among the 3 models namely Logistic regression, Support vector machine with 3 kernels and Decision tree.

We can conclude that SVM machine with poly kernel is the best fitting model for the universal bank data as it has the highest recall score when compared with Logistic regression and decision tree.